# Imports 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import pandas as pd
import numpy as np
import xarray as xr

from fastai.vision.all import *
from layers.convGRU import ConvGRU
from sklearn.metrics import r2_score as sk_r2_score

from mpl_toolkits import mplot3d
from matplotlib import cm
from matplotlib.tri import Triangulation
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [ ]:
# import wandb
# from fastai.callback.wandb import *
# os.environ['WANDB_NOTEBOOK_NAME'] = 'med_wind' 

# Download wind data 
Requires installing cdsapi from: https://cds.climate.copernicus.eu/api-how-to

This might take a while, but only need to run once.

In [ ]:
import datetime
import cdsapi
c = cdsapi.Client()

## Forecast

In [ ]:
hours=[6,18]
for year in range(2000,2019):
    for hour in hours:
        start=datetime.datetime(year-1,12,31,hour,0,0)
        end=datetime.datetime(year+1,1,1,hour,0,0)
        oneday=datetime.timedelta(1,0,0)
        current=start
        while current<end:
            c.retrieve('reanalysis-era5-complete', {
            'format':'netcdf',
            'class': 'ea',
            'date': '%4d-%02d-%02d'  %(current.year,current.month,current.day),
            'expver': '1',
            'levtype': 'sfc',
            'grid':'0.1/0.1',
            'area' : '50/-10/29/40',
            'param': '165.128/166.128',
            'step': '0/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18',
            'stream': 'oper',
            'time': '%02d:%02d:%02d' %(current.hour,current.minute,current.second),
            'type': 'fc',
            }, 'data/input/fore/era5_forecast_%4d%02d%02d%02d.nc' %(current.year,current.month,current.day,current.hour))
            current=current+oneday

## Reanalysis

In [ ]:
for year in range(2000,2019):
    c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type':'reanalysis',
            'format':'netcdf',
            'variable':[
                '10m_u_component_of_wind','10m_v_component_of_wind'
            ],
            'grid':'0.1/0.1',
            'area' : '50/-10/29/40',
            'year':'%4d' %year,
            'month':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12'
            ],
            'day':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12',
                '13','14','15',
                '16','17','18',
                '19','20','21',
                '22','23','24',
                '25','26','27',
                '28','29','30',
                '31'
            ],
            'time':[
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
            ]
        },
        'data/input/rean/era5_reanalysis_%4d.nc' %year )

# Define model

In [ ]:
class convGRU_res_model(Module):
    """
    This model builds a pseudo-Unet model using dilation, with a convGRU in the middle.
    Args:
        n_conv_hidden (list): Channels for pseuso-Unet
        n_dilation (list): Dilations for pseuso-Unet
        n_gru_hidden (int): Numer of convGRU layers. Default: 1
        kernel_size (int): Kernel size for pseuso-Unet. Default: 3
        in_channels (int): Number of channels of the input. Default: 1
        gru_in_shape (tuple): Dimensions of the image input to the convGRU layers. Default: (32,128)
    Shape:
        Input: (N, T, C, lat, lon)
        Output: (N, C, lat, lon)
        where:
            N = Batch size
            T = # of look-back points in time
            C = # of channels
            lat = # of latitude grid points
            lon = # of longitude grid points
    """
    def __init__(self, n_conv_hidden,n_dilation,n_gru_hidden=1,kernel_size=3,
                 in_channels=1,gru_in_shape=(32,128)):
        assert len(n_conv_hidden) == len(n_dilation)
        
        self.n_conv_hidden = n_conv_hidden
        self.n_conv_hidden.insert(0,in_channels)  # insert the input channels for first and last conv layers
        self.n_dilation = n_dilation
        self.n_gru_hidden = n_gru_hidden
        self.n_conv_layers = len(self.n_dilation)
        self.kernel_size = kernel_size
        self.gru_in_shape = gru_in_shape

        self.conv_in = []
        self.conv_out = []
        self.gru = []
        
        # Loop to build encoder part of pseuso-Unet
        for l in range(self.n_conv_layers):
            self.conv_in.append(
            nn.Conv2d(
                in_channels = self.n_conv_hidden[l],
                out_channels = self.n_conv_hidden[l+1],
                kernel_size = self.kernel_size,
                padding = self.kernel_size//2*self.n_dilation[l],
                dilation = self.n_dilation[l]
                ).cuda()
            )
        self.conv_in = nn.ModuleList(self.conv_in)                
        
        # Build convGRU layers
        self.gru = ConvGRU(
                input_size = self.gru_in_shape,
                input_dim = self.n_conv_hidden[-1],
                hidden_dim = self.n_conv_hidden[-1],
                kernel_size = (self.kernel_size,self.kernel_size),
                num_layers = self.n_gru_hidden,
                dtype = torch.cuda.FloatTensor,
                batch_first=True,
                bias=True,
                return_all_layers=False
                    ).cuda()
        
        # Loop to build decoder part of pseuso-Unet
        for l in range(self.n_conv_layers-1,0,-1):
            self.conv_out.append(
            nn.Conv2d(
                in_channels = self.n_conv_hidden[l+1]*2,
                out_channels = self.n_conv_hidden[l]*2,
                kernel_size = self.kernel_size,
                padding = self.kernel_size//2*self.n_dilation[l],
                dilation = self.n_dilation[l]
                )
            )        
        # last decoder layer out of loop for out_channels without the x2
        self.conv_out.append(
            nn.Conv2d(
                in_channels = self.n_conv_hidden[1]*2,
                out_channels = self.n_conv_hidden[0],
                kernel_size = self.kernel_size,
                padding = self.kernel_size//2*self.n_dilation[0],
                dilation = self.n_dilation[0]
                ).cuda()
            )        
        
        self.conv_out = nn.ModuleList(self.conv_out)

                
    def forward(self, x):
        x=x.cuda()
        inp=x[:,-1]   # residual
        seq_len = x.shape[1]
        
        # encoder
        for l in torch.arange(self.n_conv_layers, dtype=torch.int32):
            xlist = []
            for t in range(seq_len):
                xlist.append(self.conv_in[l](x[:,t]).unsqueeze(dim=1))
            x=torch.cat(xlist,dim=1)
        
        # ConvGRU
        xlist, hlist = self.gru(x.cuda())
        x=torch.cat((xlist[-1][:,-1],x[:,-1]),dim=1).cuda()
        
        # Decoder
        for l in torch.arange(self.n_conv_layers, dtype=torch.int32):
            x = self.conv_out[l](x)
        
        # Residual
        x = x+inp
        return x

# Load data

In [ ]:
data_path= Path('data/')

In [ ]:
fnames=sorted([str(x) for x in Path(data_path.as_posix()+'/input/fore/').ls()])
x_fore = xr.open_mfdataset(fnames,   #'/home/yuval/projects/data/wind/era5_forecast/*.nc', 
                               combine='by_coords', preprocess=lambda x:x.isel(time=slice(6,18)))
x_fore = x_fore.rename({'longitude':'lon','latitude':'lat'})
x_fore = x_fore.sortby('lat')
x_fore

In [ ]:
x_reanalysis = xr.open_mfdataset(data_path.as_posix()+'/input/rean/*.nc',
                                 combine='by_coords')
x_reanalysis = x_reanalysis.rename({'longitude':'lon','latitude':'lat'})
x_reanalysis = x_reanalysis.sortby('lat')
x_reanalysis

In [ ]:
def combine_fore_rean(fore,rean,params=['wnd_vel','dir']):
    "Combines given forecast and reanalysis arrays to a single array with params properties"
    fore = fore[params]
    rean = rean[params]
    for var in params:
        fore=fore.rename({var:(var+'_fore')})
        fore[var+'_rean'] = rean[var]
#         rean.rename({var:var+'_rean'},inpalce=True)
    
    return fore

def get_seq(ds,n_recent=12,wnd_norm=1,params=['wnd_vel']):
    """
    Creates a sequence from combined fore and rean array.
    Sequence is a tuple of Shape ((N, T, C, lat, lon), (N, C, lat, lon)) where:
        N = Batch size
        T = # of look-back points in time == n_recent
        C = # of channels (given by number of params)
        lat = # of latitude grid points
        lon = # of longitude grid points
    the property can be devided by a given wnd_norm. Default: 1
    """
    dt=(ds.time.values[1:]-ds.time.values[:-1]).astype('timedelta64[m]').astype('int')
    dt_median = np.median(dt).astype('int')
    dt=np.append(dt_median,dt)
    X=[]
    y=[]
    for p in params:
        X.append(torch.Tensor(ds[p+'_fore'].values/wnd_norm).unsqueeze(dim=1))
        y.append(torch.Tensor(ds[p+'_rean'].values/wnd_norm).unsqueeze(dim=1))
    X = torch.cat(X,dim=1)
    y = torch.cat(y,dim=1)
    seq = [(X[i:i+n_recent], y[i+n_recent-1]) 
           for i in range(len(X)-n_recent+1) 
           if ~(dt[i:i+n_recent] > dt_median).any()               # this removes seq with time jump
          ]
    return seq    


def add_prop(ds):
    "Adds wind magnitude (wnd_vel), direction and cos/sin of that direction to the wind array"
    ds['wnd_vel'] = (ds.u10**2+ds.v10**2)**0.5
    ds['cos'] = ds.u10/ds.wnd_vel
    ds['sin'] = ds.v10/ds.wnd_vel
    ds['dir'] = xr.ufuncs.arctan2(ds.v10,ds.u10)
    return ds



In [ ]:
"Define dimensions of wind array in space and time"
lon_min, lon_max , lat_min , lat_max = -2.1, 36.2, 30.2, 45.8
lon_slice = slice(lon_min,lon_max,3)
lat_slice = slice(lat_min,lat_max,5)
time_slice = slice('2017')
# time_slice = slice('2000', '2016')

rean = x_reanalysis.sel(lon=lon_slice,lat=lat_slice)
fore = x_fore.sel(lon=lon_slice,lat=lat_slice)
rean = add_prop(rean)
fore = add_prop(fore)

# "This could have high memory usage"
# rean=rean.load() 
# fore=fore.load()

# Prepare data and model

In [ ]:
def get_rmse_stats(val_true,val_pred,val_x,run,val_year,keep_rmses=False):
    """
    Helper function to store and print RMSE.
    Args:
        val_true (Tensor): Ground truth
        val_pred (Tensor): Model predition
        val_x (Tensor): Model input, which is the original forecast
        run (int): Run number
        val_year (int): Year of validation
        keep_rmses (Bool): If this is true the RMSE raw values are stored. Default: False
    """
    rmses=pd.DataFrame(columns=['rean-fore', 'rean-pred'])
    for i in range(len(val_pred)):
        rmses=rmses.append({'rean-fore':rmse(val_true[i],val_x[i]).numpy(), 
                            'rean-pred':rmse(val_true[i],val_pred[i]).numpy()},
                           ignore_index=True)
    
    print('RMSE rean-pred: %f\nRMSE rean-fore: %f\nRMSE improved by %f\n' 
      %(rmse(val_true,val_pred),
        rmse(val_true,val_x),
       1-rmse(val_true,val_pred)/rmse(val_true,val_x)))
    print('max RMSE pred: %f, fore: %f\nmin RMSE pred: %f, fore: %f\nmean RMSE pred: %f, fore: %f\nmedian RMSE pred: %f, fore: %f\n'
          %(rmses.max()[1],rmses.max()[0],
           rmses.min()[1],rmses.min()[0],
           rmses.mean()[1],rmses.mean()[0],
           rmses.median()[1],rmses.median()[0]))
    
    dict={
       'RMSE rean-pred': rmse(val_true,val_pred).unsqueeze(dim=0).numpy()[0],
       'RMSE rean-fore': rmse(val_true,val_x).unsqueeze(dim=0).numpy()[0],
       'RMSE improved by': (1-rmse(val_true,val_pred)/rmse(val_true,val_x)).unsqueeze(dim=0).numpy()[0],
        'pred max rmse': rmses.max()[1],
        'fore max rmse': rmses.max()[0],
        'pred min rmse': rmses.min()[1],
        'fore min rmse': rmses.min()[0],
        'pred mean rmse': rmses.mean()[1],
        'fore mean rmse': rmses.mean()[0],
        'pred median rmse': rmses.median()[1], 
        'fore median rmse': rmses.median()[0],
        'val year': val_year
    }
    
    if keep_rmses:
        dict['rmses'] = rmses
    
    return dict

In [ ]:
def run_cycles(runs,epochs=2,lr=1e-3,dev=1,pct=0.5,name='UMag',save=False):
    """
    Runs model with changing learning rate (LR), saves and logs statistics
    Args:
        runs (int): # of runs with different LR
        epochs (int): # of epochs per run. Default: 2
        lr (int): Initial LR. Default: 1e-3
        dev (int): LR devider, with power of current run. Default: 1
        pct (int): One cycle PCT. Default: 0.5
        name (string): Model name. Default: 'UMag'
        save (Bool): If true model is saved after each run. Default: False
    """
    for run in range(runs):
        print('run_%s_%dcyc' %(name,(epochs*(run+1))))
        learn.fit_one_cycle(epochs, lr/(dev**run), pct_start=pct)
        if save:
            learn.save('run_%s_%dcyc' %(name,(epochs*(run+1))))
        val_pred,val_true = learn.get_preds()
        val_x=torch.cat([x[0][-1:] for x in dls.valid_ds],dim=0)
        rmse_dict = get_rmse_stats(val_true,val_pred,val_x,'1',2017)
#         wandb.log(rmse_dict)

## UMag

In [ ]:
"This section runs the model for UMag, meaning it takes the wind magnitude as input and returns the same"

In [ ]:
params=['wnd_vel']
data = combine_fore_rean(fore,rean,params)

pdtimes = pd.DatetimeIndex(data.time.values)
val = data.sel(time=slice('2017-01-02 08','2017-01-02 22'))
# val = data.sel(time='2000')
val = val.load()

train = data.sel(time=slice('2017-01-01 12','2017-01-02 07'))
# train = data.sel(time=slice('2001','2016'))
train = train.load()

seq = get_seq(train,params=params)
seq_val = get_seq(val,params=params)

# bs=16
bs=4
dls = DataLoaders.from_dsets(seq,seq_val,bs=bs,
                             shuffle=False, drop_last=True)

arch = convGRU_res_model([8,16,64,128],[1,2,4,8],1,in_channels=1).cuda()

learn = Learner(dls, arch, wd=1e-3, loss_func=MSELossFlat(),metrics=rmse
#                ,cbs=WandbCallback()              
               )

In [ ]:
run_cycles(2,epochs=2,lr=1e-3,dev=10,pct=0.3,save=True)

## UVec

In [ ]:
"This section runs the model for UVec, meaning it takes the wind vector (u10,v10) as input and returns the same"

In [ ]:
params=['u10','v10']
data = combine_fore_rean(fore,rean,params)

pdtimes = pd.DatetimeIndex(data.time.values)
val = data.sel(time=slice('2017-01-02 08','2017-01-02 22'))
# val = data.sel(time='2000')
val = val.load()

train = data.sel(time=slice('2017-01-01 12','2017-01-02 07'))
# train = data.sel(time=slice('2001','2016'))
train = train.load()

seq = get_seq(train,params=params)
seq_val = get_seq(val,params=params)

# bs=16
bs=4
dls = DataLoaders.from_dsets(seq,seq_val,bs=bs,
                             shuffle=False, drop_last=True)

arch = convGRU_res_model([8,16,64,128],[1,2,4,8],1,in_channels=2).cuda()

learn = Learner(dls, arch, wd=1e-3, loss_func=MSELossFlat(),metrics=rmse
#                ,cbs=WandbCallback()              
               )

In [ ]:
run_cycles(2,epochs=2,lr=1e-3,dev=10,pct=0.3,save=True,name='UVec')

# Generate wind.nc files 
this creates .nc files from the deep learning model to be used in WAVEWATCH III wave forecasting model

In [ ]:
def relative_error(old,new):
    return (np.abs((old - new) / old)).mean()

In [ ]:
data_test = combine_fore_rean(fore,rean,['wnd_vel','cos','sin','dir','u10','v10'])
time_slice =slice('2017-01-02 23','2017-01-03 11')
# time_slice = '2017'
data_test = data_test.sel(time=time_slice)
data_test=data_test.load()
seq_test = get_seq(data_test)

## UMag 

In [ ]:
seq_test = get_seq(data_test)
arch = convGRU_res_model([8,16,64,128],[1,2,4,8],1,in_channels=1).cuda()
dls_test = DataLoaders.from_dsets(seq_test,seq_test,bs=bs,
                                 shuffle=False, drop_last=True)
learn_test = Learner(dls_test, arch, wd=1e-3, loss_func=MSELossFlat(),metrics=rmse)
# learn_test.path=Path('/home/yuval/projects/wave_grid/era5_wind')
learn_test.load('run_UMag_4cyc');

In [ ]:
test_pred,test_true = learn_test.get_preds()
test_x=torch.cat([x[0][-1:] for x in dls_test.valid_ds],dim=0)
test_pred = test_pred/2  # this is a fix as it was trained with mean instead of sum

In [ ]:
print('wnd_vel RMSE pred: %f,   fore: %f,  improved by: %f' %(rmse(test_true,test_pred),rmse(test_true,test_x),
                                            1-rmse(test_true,test_pred)/rmse(test_true,test_x)))

In [ ]:
# this is for UMag prediction to u10/v10 with original direction

#FIX TIME TO MATCH PREDICTION
max_time_idx = len(data_test.time)
n_recent=12
data_test_cut = data_test.copy().isel(time=slice(n_recent-1,max_time_idx))
assert len(data_test_cut.time) == len(test_pred)

ds_test_fore=xr.Dataset({
    'u10': (['time','lat','lon'],data_test_cut.cos_fore.values*test_x.squeeze().numpy()),
    'v10': (['time','lat','lon'],data_test_cut.sin_fore.values*test_x.squeeze().numpy())},
    coords={
        'time': data_test_cut.time.values,
        'lat': data_test_cut.lat.values,
        'lon': data_test_cut.lon.values
    })

ds_test_pred=xr.Dataset({
    'u10': (['time','lat','lon'],data_test_cut.cos_fore.values*test_pred.squeeze().numpy()),
    'v10': (['time','lat','lon'],data_test_cut.sin_fore.values*test_pred.squeeze().numpy())},
    coords={
        'time': data_test_cut.time.values,
        'lat': data_test_cut.lat.values,
        'lon': data_test_cut.lon.values
    })

ds_test_rean=xr.Dataset({
    'u10': (['time','lat','lon'],data_test_cut.cos_rean.values*test_true.squeeze().numpy()),
    'v10': (['time','lat','lon'],data_test_cut.sin_rean.values*test_true.squeeze().numpy())},
    coords={
        'time': data_test_cut.time.values,
        'lat': data_test_cut.lat.values,
        'lon': data_test_cut.lon.values
    })

# test relative error of putting the data through the proccessing (should be close to zero)
assert relative_error(data_test_cut.u10_fore, ds_test_fore.u10) < 1e-8
assert relative_error(data_test_cut.v10_fore, ds_test_fore.v10) < 1e-8
assert relative_error(data_test_cut.u10_rean, ds_test_rean.u10) < 1e-8
assert relative_error(data_test_cut.v10_rean, ds_test_rean.v10) < 1e-8

In [ ]:
print('u10 RMSE pred: %f,   fore: %f,  improved by: %f' %(
    rmse(torch.Tensor(ds_test_rean.u10.values),torch.Tensor(ds_test_pred.u10.values)),
    rmse(torch.Tensor(ds_test_rean.u10.values),torch.Tensor(ds_test_fore.u10.values)),
1-rmse(torch.Tensor(ds_test_rean.u10.values),torch.Tensor(ds_test_pred.u10.values))/
rmse(torch.Tensor(ds_test_rean.u10.values),torch.Tensor(ds_test_fore.u10.values))))

print('v10 RMSE pred: %f,   fore: %f,  improved by: %f' %(
    rmse(torch.Tensor(ds_test_rean.v10.values),torch.Tensor(ds_test_pred.v10.values)),
    rmse(torch.Tensor(ds_test_rean.v10.values),torch.Tensor(ds_test_fore.v10.values)),
1-rmse(torch.Tensor(ds_test_rean.v10.values),torch.Tensor(ds_test_pred.v10.values))/
rmse(torch.Tensor(ds_test_rean.v10.values),torch.Tensor(ds_test_fore.v10.values))))

In [ ]:
ds_test_fore.to_netcdf('data/output/era5_test_forecast.nc')
ds_test_pred.to_netcdf('data/output/era5_test_umag.nc')
ds_test_rean.to_netcdf('data/output/era5_test_reanalysis.nc')

## UVec

In [ ]:
params=['u10','v10']
seq_test = get_seq(data_test,params=params)
arch = convGRU_res_model([8,16,64,128],[1,2,4,8],1,in_channels=len(params)).cuda()
dls_test = DataLoaders.from_dsets(seq_test,seq_test,bs=bs,
                                 shuffle=False, drop_last=True)

learn_test = Learner(dls_test, arch, wd=1e-3, loss_func=MSELossFlat(),metrics=rmse)
learn_test.load('run_UVec_4cyc');

In [ ]:
test_pred,test_true = learn_test.get_preds()
test_x=torch.cat([x[0][-1:] for x in dls_test.valid_ds],dim=0)

In [ ]:
print('u10 RMSE pred: %f,   fore: %f,  improved by: %f' %(rmse(test_true[:,0],test_pred[:,0]),
        rmse(test_true[:,0],test_x[:,0]),1-rmse(test_true[:,0],test_pred[:,0])/rmse(test_true[:,0],test_x[:,0])))
print('v10 RMSE pred: %f,   fore: %f,  improved by: %f' %(rmse(test_true[:,1],test_pred[:,1]),
        rmse(test_true[:,1],test_x[:,1]),1-rmse(test_true[:,1],test_pred[:,1])/rmse(test_true[:,1],test_x[:,1])))

In [ ]:
#FIX TIME TO MATCH PREDICTION!
max_time_idx = len(data_test.time)
n_recent=12
data_test_cut = data_test.copy().isel(time=slice(n_recent-1,max_time_idx))
assert len(data_test_cut.time) == len(test_pred)

ds_test_fore=xr.Dataset({
    'u10': (['time','lat','lon'],test_x[:,0].numpy()),
    'v10': (['time','lat','lon'],test_x[:,1].numpy())},
    coords={
        'time': data_test_cut.time.values,
        'lat': data_test_cut.lat.values,
        'lon': data_test_cut.lon.values
    })

ds_test_pred=xr.Dataset({
    'u10': (['time','lat','lon'],test_pred[:,0].numpy()),
    'v10': (['time','lat','lon'],test_pred[:,1].numpy())},
    coords={
        'time': data_test_cut.time.values,
        'lat': data_test_cut.lat.values,
        'lon': data_test_cut.lon.values
    })

ds_test_rean=xr.Dataset({
    'u10': (['time','lat','lon'],test_true[:,0].numpy()),
    'v10': (['time','lat','lon'],test_true[:,1].numpy())},
    coords={
        'time': data_test_cut.time.values,
        'lat': data_test_cut.lat.values,
        'lon': data_test_cut.lon.values
    })

# test relative error of putting the data through the proccessing (should be close to zero)
assert relative_error(data_test_cut.u10_fore, ds_test_fore.u10) < 1e-8
assert relative_error(data_test_cut.v10_fore, ds_test_fore.v10) < 1e-8
assert relative_error(data_test_cut.u10_rean, ds_test_rean.u10) < 1e-8
assert relative_error(data_test_cut.v10_rean, ds_test_rean.v10) < 1e-8

In [ ]:
print('u10 RMSE pred: %f,   fore: %f,  improved by: %f' %(
    rmse(torch.Tensor(ds_test_rean.u10.values),torch.Tensor(ds_test_pred.u10.values)),
    rmse(torch.Tensor(ds_test_rean.u10.values),torch.Tensor(ds_test_fore.u10.values)),
1-rmse(torch.Tensor(ds_test_rean.u10.values),torch.Tensor(ds_test_pred.u10.values))/
rmse(torch.Tensor(ds_test_rean.u10.values),torch.Tensor(ds_test_fore.u10.values))))

print('v10 RMSE pred: %f,   fore: %f,  improved by: %f' %(
    rmse(torch.Tensor(ds_test_rean.v10.values),torch.Tensor(ds_test_pred.v10.values)),
    rmse(torch.Tensor(ds_test_rean.v10.values),torch.Tensor(ds_test_fore.v10.values)),
1-rmse(torch.Tensor(ds_test_rean.v10.values),torch.Tensor(ds_test_pred.v10.values))/
rmse(torch.Tensor(ds_test_rean.v10.values),torch.Tensor(ds_test_fore.v10.values))))

In [ ]:
ds_test_pred.to_netcdf('data/output/era5_test_uvec.nc')

# Stats and plots

In [ ]:
plots_path= Path('plots/')
wind_path = Path('data/output/')

## Wind

In [ ]:
wind_fore = xr.open_dataset(wind_path.as_posix()+'/era5_test_forecast.nc')
wind_rean = xr.open_mfdataset(wind_path.as_posix()+'/era5_test_reanalysis.nc')
wind_umag = xr.open_mfdataset(wind_path.as_posix()+'/era5_test_umag.nc')
wind_uvec = xr.open_mfdataset(wind_path.as_posix()+'/era5_test_uvec.nc')

In [ ]:
wind_fore = add_prop(wind_fore)
wind_rean = add_prop(wind_rean)
wind_umag = add_prop(wind_umag)
wind_uvec = add_prop(wind_uvec)

### Helper functions

In [ ]:
def ds_rmse(ds_pred,ds_true=wind_rean,var='wnd_vel',norm=False):
    """
    Calculates RMSE from two xarrays for given variable.
    Args:
        ds_pred (xarray): Prediction xarray.
        ds_true (xarray): Ground truth xarray. Default: wind_rean
        var (str): Variable to calculate RMSE for. Default: wnd_vel
        norm (Bool): Normalize the error by deviding it with the GT if true. Default: False
    """
    err=ds_true[var].values-ds_pred.sel(time=ds_true.time.values)[var].values
    if reg:
        err /= ds_true[var].values
    err = np.nan_to_num(err)
    return (err**2).mean(axis=dim)**0.5


def ds_bias(ds_pred,ds_true=wind_rean,var='wnd_vel',norm=False):
    err=ds_pred[var].values-ds_true[var].values
    if norm:
        err /= ds_true[var].values
    err = err[~np.isnan(err)]
    return (err).mean()

def ds_SI(ds_pred,ds_true=wind_rean,var='wnd_vel'):
    pred=ds_pred[var].values
    true=ds_true[var].values
    cond=np.isnan(pred)+np.isnan(true)
    pred = pred[~cond]
    true = true[~cond]
    si=(((pred-pred.mean())-(true-true.mean()))**2).mean()**0.5 / true.mean() * 100
#     si = si[~np.isnan(si)]
    return si

def ds_R2(ds_pred,ds_true=wind_rean,var='wnd_vel'):
    pred=ds_pred[var].values
    true=ds_true[var].values
    cond=np.isnan(pred)+np.isnan(true)
    pred = pred[~cond]
    true = true[~cond]
    r2 = sk_r2_score(pred,true)
#     r2=r2_score(torch.Tensor(pred),torch.Tensor(true))
    return r2

def ds_diurnal_cycle_mean(ds_pred,var='wnd_vel', with_hour=False):
    """
    Returns the mean diurnal cycle from an xarray for given variable.
    Args:
        ds_pred (xarray): Xarray input.
        var (str): Variable to calculate mean diurnal cycle for. Default: wnd_vel
        with_hour (Bool): If true returns a tuple of (hours,mean diurnal cycle). Defalut: False
    """
    pred = ds_pred.groupby("time.hour").mean()[var].values
    if with_hour:
        return ds_pred.groupby("time.hour").mean().hour,(pred).mean(axis=1).mean(axis=1)
    return (pred).mean(axis=1).mean(axis=1)

In [ ]:
def get_stats(pred,func=ds_rmse,rean=wind_rean):
    u10=func(pred,ds_true=rean,var='u10')
    v10=func(pred,ds_true=rean,var='v10')
    U=func(pred,ds_true=rean,var='wnd_vel')
    cos=func(pred,ds_true=rean,var='cos')
    sin=func(pred,ds_true=rean,var='sin')
    return U,u10,v10,cos,sin

### stats

In [ ]:
pd_indx = [
            ['RMSE','RMSE','RMSE',
             'Bias','Bias','Bias',
             'SI','SI','SI',
             'R2','R2','R2'
#              ,'Dcyc','Dcyc','Dcyc'
           ],
    ['U[m/s]','u10[m/s]','v10[m/s]'
    ,'U[m/s]','u10[m/s]','v10[m/s]'
    ,'U[m/s]','u10[m/s]','v10[m/s]'
    ,'U[m/s]','u10[m/s]','v10[m/s]'
#     ,'U[m/s]','u10[m/s]','v10[m/s]'
    ]]
pd_indx = list(zip(*pd_indx))
pd_indx = pd.MultiIndex.from_tuples(pd_indx, names=['Stat', 'Property'])


In [ ]:
fc_data = np.concatenate((np.array(get_stats(wind_fore,func=ds_rmse)[:3]),
                                           np.array(get_stats(wind_fore,func=ds_bias)[:3]),
                                           np.array(get_stats(wind_fore,func=ds_SI)[:3]),
                                           np.array(get_stats(wind_fore,func=ds_R2)[:3])
#                                            ,np.array(get_stats(wind_fore,func=ds_diurnal_cycle_error)[:3])
                                            )) 

umag_data = np.concatenate((np.array(get_stats(wind_umag,func=ds_rmse)[:3]),
                                           np.array(get_stats(wind_umag,func=ds_bias)[:3]),
                                           np.array(get_stats(wind_umag,func=ds_SI)[:3]),
                                           np.array(get_stats(wind_umag,func=ds_R2)[:3])
#                                            ,np.array(get_stats(wind_umag,func=ds_diurnal_cycle_error)[:3])
                                            )) 

uvec_data = np.concatenate((np.array(get_stats(wind_uvec,func=ds_rmse)[:3]),
                                           np.array(get_stats(wind_uvec,func=ds_bias)[:3]),
                                           np.array(get_stats(wind_uvec,func=ds_SI)[:3]),
                                           np.array(get_stats(wind_uvec,func=ds_R2)[:3])
#                                            ,np.array(get_stats(wind_uvec,func=ds_diurnal_cycle_error)[:3])
                                            )) 

In [ ]:
df = pd.DataFrame(data={
#                         'FC':fc_data ,'UMag': umag_data ,'UVec': uvec_data
                        'FC':np.around(fc_data,4),
                        'UMag': np.around(umag_data,4), 'UMag imp.[%]':np.around((1-np.abs(umag_data/fc_data))*100,1),
                        'UVec': np.around(uvec_data,4), 'UVec imp.[%]':np.around((1-np.abs(uvec_data/fc_data))*100,1)
                       }
                  , index=pd_indx)
df

### Plots

In [ ]:
def wind_rmse(rean,pred,dim=0):
    """
    Returns RMSE map, with the mean only on defined dim.
    Args:
        rean (Tensor): Ground truth.
        pred (Tensor): Prediction
        dim (int): Dimension for mean. Default: 0
    """
    return ((rean-pred)**2).mean(dim=dim)**0.5
    
def wind_sp_rmses(pred,rean=wind_rean):
    "Return spatial RMSE maps of u10,v10,U for given xarrays"
    u10=wind_rmse(torch.Tensor(rean.u10.values),torch.Tensor(pred.u10.values))
    v10=wind_rmse(torch.Tensor(rean.v10.values),torch.Tensor(pred.v10.values))
    Urean=(rean.u10.values**2+rean.v10.values**2)**0.5
    Upred=(pred.u10.values**2+pred.v10.values**2)**0.5
    U=wind_rmse(torch.Tensor(Urean),torch.Tensor(Upred))
    
    return u10,v10,U

In [ ]:
wind_fore_rmses = wind_sp_rmses(wind_fore)
wind_umag_rmses = wind_sp_rmses(wind_umag)
wind_uvec_rmses = wind_sp_rmses(wind_uvec)

In [ ]:
"This creates a basemap of the shoreline"
lons, lats = np.meshgrid(wind_rean.lon.values,wind_rean.lat.values)
m = Basemap(
            projection = 'merc',
            llcrnrlat=lats.min(), urcrnrlat=lats.max(),
            llcrnrlon=lons.min(), urcrnrlon=lons.max(),
            resolution='h', area_thresh=100
        )

X,Y = m(lons, lats)

In [ ]:
def plot_winds(pred,name,save=None,fore=wind_fore_rmses,vmax=0.10,figsize=(8,17),cmap='seismic',m=m,cdens=200,dpi=100.0):
    """
    Plots RMSE difference of u10,v10,U.
    Args:
        pred (tuple): Tuple of prediction u10,v10,U RMSEs.
        name (str): Model name.
        save (str): If not none, saves plot as jpeg and eps by this name. Default: None
        fore (tuple): Tuple of prediction u10,v10,U RMSEs. Default: wind_fore_rmses
        vmax (int): Min/max value for z-axis. Default: 0.10
        figsize (tuple): Default: (8,17)
        cmap (str): cmap. Default: 'seismic'
        m (basemap): Basemap for shoreline. Default: m
        cdens (int):  cdens. Default: 200
        dpi (float): dpi. Default: 100.0
    """
    Z0 = fore[0]-pred[0]
    Z1 = fore[1]-pred[1]
    Z2 = fore[2]-pred[2]

    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(311)
    m.drawcoastlines(linewidth=0.5)
    # m.drawcountries()
    cs = m.contourf(X,Y,Z0, cdens, vmin=-vmax, vmax=vmax, cmap=cmap)
    m.drawparallels(np.linspace(31,45,5), labels=[True, False, False, False], linewidth=0.0)
    m.drawmeridians(np.linspace(-2,36,5), labels=[False, False, False, True], linewidth=0.0)
    cbar_clr = plt.cm.ScalarMappable(cmap=cmap)
    cbar_clr.set_array(Z0)
    cbar_clr.set_clim(-vmax, vmax)
    cbar = m.colorbar(cbar_clr, location='bottom',pad='10%')
    cbar.ax.set_xlabel('u10 RMSE [m/s]', rotation=0)
    ax.set_title('u10 FC RMSE - %s RMSE' %name)

    ax = fig.add_subplot(312)
    m.drawcoastlines(linewidth=0.5)
    # m.drawcountries()
    cs = m.contourf(X,Y,Z1, cdens, vmin=-vmax, vmax=vmax, cmap=cmap)
    m.drawparallels(np.linspace(31,45,5), labels=[True, False, False, False], linewidth=0.0)
    m.drawmeridians(np.linspace(-2,36,5), labels=[False, False, False, True], linewidth=0.0)
    cbar_clr = plt.cm.ScalarMappable(cmap=cmap)
    cbar_clr.set_array(Z1)
    cbar_clr.set_clim(-vmax, vmax)
    cbar = m.colorbar(cbar_clr, location='bottom',pad='10%')
    cbar.ax.set_xlabel('v10 RMSE [m/s]', rotation=0)
    ax.set_title('v10 FC RMSE - %s RMSE' %name)

    ax = fig.add_subplot(313)
    m.drawcoastlines(linewidth=0.5)
    # m.drawcountries()
    cs = m.contourf(X,Y,Z2, cdens, vmin=-vmax, vmax=vmax, cmap=cmap)
    m.drawparallels(np.linspace(31,45,5), labels=[True, False, False, False], linewidth=0.0)
    m.drawmeridians(np.linspace(-2,36,5), labels=[False, False, False, True], linewidth=0.0)
    cbar_clr = plt.cm.ScalarMappable(cmap=cmap)
    cbar_clr.set_array(Z2)
    cbar_clr.set_clim(-vmax, vmax)
    cbar = m.colorbar(cbar_clr, location='bottom',pad='10%')
    cbar.ax.set_xlabel('U RMSE [m/s]', rotation=0)
    ax.set_title('U FC RMSE - %s RMSE' %name)
    
    plt.tight_layout()
    plt.show() 
    
    if save is not None:
        fig.savefig(paper_path.as_posix()+'/wind/'+save+'.eps', format='eps')
        fig.savefig(paper_path.as_posix()+'/wind/'+save+'.jpg', format='jpeg')

In [ ]:
plot_winds(wind_umag_rmses,'UMag', figsize=(7.2,14), dpi=100
#           ,save='UMag_winds_spatial'
          )

In [ ]:
plot_winds(wind_uvec_rmses,'UVec', figsize=(7.2,14), dpi=100
#           ,save='UVec_winds_spatial'
          )


#### Diurnal cycle

In [ ]:
# ds_diurnal_cycle_error(wind_fore), ds_diurnal_cycle_error(wind_umag), ds_diurnal_cycle_error(wind_uvec)
fig = plt.figure()
plt.plot(*ds_diurnal_cycle_mean(wind_rean,with_hour=True))
plt.plot(*ds_diurnal_cycle_mean(wind_fore,with_hour=True),'.')
plt.plot(*ds_diurnal_cycle_mean(wind_umag,with_hour=True),'r*')
plt.plot(*ds_diurnal_cycle_mean(wind_uvec,with_hour=True),'go')
plt.legend(('REAN','FC','UMag','UVec'))
plt.xlabel('Hour')
plt.ylabel('U[m/s]')
plt.title('Wind Velocity Diurnal Cycle')

# fig.savefig(paper_path.as_posix() + '/Dcyc_wind_vel.eps', format='eps')

In [ ]:
# ds_diurnal_cycle_error(wind_fore), ds_diurnal_cycle_error(wind_umag), ds_diurnal_cycle_error(wind_uvec)
fig = plt.figure()
plt.plot(*ds_diurnal_cycle_mean(wind_rean,with_hour=True,var='u10'))
plt.plot(*ds_diurnal_cycle_mean(wind_fore,with_hour=True,var='u10'),'.')
plt.plot(*ds_diurnal_cycle_mean(wind_umag,with_hour=True,var='u10'),'r*')
plt.plot(*ds_diurnal_cycle_mean(wind_uvec,with_hour=True,var='u10'),'go')
plt.legend(('REAN','FC','UMag','UVec'))
plt.xlabel('Hour')
plt.ylabel('u10[m/s]')
plt.title('u10 Wind Velocity Diurnal Cycle')

# fig.savefig(paper_path.as_posix() + '/Dcyc_u10.eps', format='eps')

In [ ]:
# ds_diurnal_cycle_error(wind_fore), ds_diurnal_cycle_error(wind_umag), ds_diurnal_cycle_error(wind_uvec)
fig = plt.figure()
plt.plot(*ds_diurnal_cycle_mean(wind_rean,with_hour=True,var='v10'))
plt.plot(*ds_diurnal_cycle_mean(wind_fore,with_hour=True,var='v10'),'.')
plt.plot(*ds_diurnal_cycle_mean(wind_umag,with_hour=True,var='v10'),'r*')
plt.plot(*ds_diurnal_cycle_mean(wind_uvec,with_hour=True,var='v10'),'go')
plt.legend(('REAN','FC','UMag','UVec'))
plt.xlabel('Hour')
plt.ylabel('v10[m/s]')
plt.title('v10 Wind Velocity Diurnal Cycle')

# fig.savefig(paper_path.as_posix() + '/Dcyc_v10.eps', format='eps')

## Waves

In [ ]:
ww3_path = Path('data/ww3/')

In [ ]:
ww3_fore = xr.open_mfdataset(ww3_path.as_posix()+'/oww3.forecast*.nc',
                                 combine='by_coords')
ww3_rean = xr.open_mfdataset(ww3_path.as_posix()+'/oww3.reanalysis*.nc',
                                 combine='by_coords')
ww3_umag = xr.open_mfdataset(ww3_path.as_posix()+'/oww3.umag*.nc',
                                 combine='by_coords')
ww3_uvec = xr.open_mfdataset(ww3_path.as_posix()+'/oww3.uvec*.nc',
                                 combine='by_coords')

### Stats

In [ ]:
def get_ww3_rmses(pred,rean=ww3_rean,params=['hs','dir','t0m1']):
    rmses = []
    for var in params:
        rmses.append(ds_rmse(pred,rean,var))
    return rmses

In [ ]:
ww3_fore_rmses = get_ww3_rmses(ww3_fore)
ww3_umag_rmses = get_ww3_rmses(ww3_umag)
ww3_uvec_rmses = get_ww3_rmses(ww3_uvec)

In [ ]:
def print_ww3_rmses(pred,name,fore=ww3_fore_rmses, params=['hs','dir','t0m1']):
    for i, var in enumerate(params):
        print('%s %s RMSE pred: %f,   fore: %f,  improved by: %f' %(name, var, pred[i],
            fore[i],1-pred[i]/fore[i]))

In [ ]:
print_ww3_rmses(ww3_umag_rmses,'UMag')
print_ww3_rmses(ww3_uvec_rmses,'UVec')

### Plots

In [ ]:
def spatial_rmse(pred,var='hs',rean=ww3_rean):
    """
    Returns RMSE map, with the mean time.
    Args:
        pred (xarray): Prediction.
        var (str): Dimension for mean. Default: 'hs'
        rean (xarray): Ground truth. Default: ww3_rean
    """
    return (((rean-pred)[var]**2).mean('time'))**0.5

In [ ]:
X = ww3_rean.isel(time=0).longitude.values
Y = ww3_rean.isel(time=0).latitude.values
tri = ww3_rean.isel(time=0).tri.values-1   # -1 so count will start from zero
triang = Triangulation(X,Y,triangles=tri)

In [ ]:
ww3_sp_fore = spatial_rmse(ww3_fore).load()
ww3_sp_umag = spatial_rmse(ww3_umag).load()
ww3_sp_uvec = spatial_rmse(ww3_uvec).load()

In [ ]:
vm = 0.12
fig, ax = plt.subplots(3,sharex=True,figsize=(8,9))
im = ax[0].tripcolor(triang,np.nan_to_num(ww3_sp_fore),vmax=vm,vmin=0)
ax[0].title.set_text('Significant wave height FC RMSE')
ax[1].tripcolor(triang,np.nan_to_num(ww3_sp_umag),vmax=vm,vmin=0)
ax[1].title.set_text('Significant wave height UMag RMSE')
ax[2].tripcolor(triang,np.nan_to_num(ww3_sp_uvec),vmax=vm,vmin=0)
ax[2].title.set_text('Significant wave height UVec RMSE')

plt.xlim((9.75,36.5))

ax[0].set_ylabel('Latitude')
ax[1].set_ylabel('Latitude')
ax[2].set_ylabel('Latitude')
ax[2].set_xlabel('Longitude')

fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.885, 0.11, 0.02, 0.77])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.ax.set_ylabel('RMSE [m]', rotation=270, labelpad=10)

# fig.savefig(paper_path.as_posix() + '/Hs_rmse.eps', format='eps')

In [ ]:
fig, ax = plt.subplots(2,sharex=True,figsize=(8,6))
im = ax[0].tripcolor(triang,np.nan_to_num(ww3_sp_fore-ww3_sp_umag),vmax=0.03,vmin=-0.03,cmap='seismic')
ax[0].title.set_text('Significant wave height FC RMSE - UMag RMSE')
ax[1].tripcolor(triang,np.nan_to_num(ww3_sp_fore-ww3_sp_uvec),vmax=0.03,vmin=-0.03,cmap='seismic')
ax[1].title.set_text('Significant wave height FC RMSE - UVec RMSE')

plt.xlim((9.75,36.5))

ax[1].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')
ax[1].set_ylabel('Latitude')


fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.885, 0.11, 0.02, 0.77])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.ax.set_ylabel('RMSE [m]', rotation=270)


# fig.savefig(paper_path.as_posix() + '/Hs_rmse_diff.eps', format='eps')

In [ ]:
var = 'dir'
vmin,vmax = -20,20
ww3_sp_fore = spatial_rmse(ww3_fore, var=var)
ww3_sp_umag = spatial_rmse(ww3_umag, var=var)
ww3_sp_uvec = spatial_rmse(ww3_uvec, var=var)

# fig,ax = plt.subplot(2)
fig, ax = plt.subplots(2,sharex=True,figsize=(8,6))
im = ax[0].tripcolor(triang,np.nan_to_num(ww3_sp_fore-ww3_sp_umag),vmax=vmax,vmin=vmin,cmap='seismic')
ax[0].title.set_text('Mean wave direction FC RMSE - UMag RMSE')
ax[1].tripcolor(triang,np.nan_to_num(ww3_sp_fore-ww3_sp_uvec),vmax=vmax,vmin=vmin,cmap='seismic')
ax[1].title.set_text('Mean wave direction FC RMSE - UVec RMSE')

plt.xlim((9.75,36.5))

ax[1].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')
ax[1].set_ylabel('Latitude')


fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.885, 0.11, 0.02, 0.77])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.ax.set_ylabel('RMSE [deg]', rotation=270)


# fig.savefig(paper_path.as_posix() + '/dir_rmse_diff.eps', format='eps')


In [ ]:
var = 't0m1'
vmin,vmax = -0.15,0.15
ww3_sp_fore = spatial_rmse(ww3_fore, var=var)
ww3_sp_umag = spatial_rmse(ww3_umag, var=var)
ww3_sp_uvec = spatial_rmse(ww3_uvec, var=var)

# fig,ax = plt.subplot(2)
fig, ax = plt.subplots(2,sharex=True,figsize=(8,6))
im = ax[0].tripcolor(triang,np.nan_to_num(ww3_sp_fore-ww3_sp_umag),vmax=vmax,vmin=vmin,cmap='seismic')
ax[0].title.set_text('Mean wave period FC RMSE - UMag RMSE')
ax[1].tripcolor(triang,np.nan_to_num(ww3_sp_fore-ww3_sp_uvec),vmax=vmax,vmin=vmin,cmap='seismic')
ax[1].title.set_text('Mean wave period FC RMSE - UVec RMSE')

plt.xlim((9.75,36.5))

ax[1].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')
ax[1].set_ylabel('Latitude')


fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.885, 0.11, 0.02, 0.77])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.ax.set_ylabel('RMSE [sec]', rotation=270)


# fig.savefig(paper_path.as_posix() + '/t0m1_rmse_diff.eps', format='eps')

### Aegean

In [ ]:
lon_ae=(22.5,29)
lat_ae=(34,41.5)
aegean_coord=np.where((ww3_rean.longitude.values>lon_ae[0]) * (ww3_rean.longitude.values<lon_ae[1]) *
                      (ww3_rean.latitude.values>lat_ae[0]) * (ww3_rean.latitude.values<lat_ae[1]) )[0]

In [ ]:
ae_fore = ww3_fore.isel(node=aegean_coord).copy()
ae_umag = ww3_umag.isel(node=aegean_coord).copy()
ae_uvec = ww3_uvec.isel(node=aegean_coord).copy()
ae_rean = ww3_rean.isel(node=aegean_coord).copy()

In [ ]:
def get_ww3_rmses(pred,rean=ww3_rean,params=['hs','dir','t0m1']):
    rmses = []
    for var in params:
        rmses.append(ds_rmse(pred,rean,var))
    return rmses

In [ ]:
umag_ae_rmses = get_ww3_rmses(ae_umag,ae_rean)
uvec_ae_rmses = get_ww3_rmses(ae_uvec,ae_rean)
fore_ae_rmses = get_ww3_rmses(ae_fore,ae_rean)

In [ ]:
def print_ww3_rmses(pred,name,fore=fore_ae_rmses, params=['hs','dir','t0m1']):
    for i, var in enumerate(params):
        print('%s %s RMSE pred: %f,   fore: %f,  improved by: %f' %(name, var, pred[i],
            fore[i],1-pred[i]/fore[i]))

In [ ]:
print_ww3_rmses(umag_ae_rmses,'UMag')
print_ww3_rmses(uvec_ae_rmses,'UVec')

In [ ]:
etesian_time = slice('15/05/2017','15/09/2017')
params = ['hs','dir','t0m1']
umag_etesian_rmses = get_ww3_rmses(ae_umag.sel(time=etesian_time),ae_rean.sel(time=etesian_time),params)
uvec_etesian_rmses = get_ww3_rmses(ae_uvec.sel(time=etesian_time),ae_rean.sel(time=etesian_time),params)
fore_etesian_rmses = get_ww3_rmses(ae_fore.sel(time=etesian_time),ae_rean.sel(time=etesian_time),params)

print_ww3_rmses(umag_etesian_rmses, 'Etesian UMag', fore_etesian_rmses, params)
print_ww3_rmses(uvec_etesian_rmses, 'Etesian UVec', fore_etesian_rmses, params)